<p style="float: left;"><a href="polymorphic-methods.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="variances.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Upper Type Bounds

In Scala, [type variables](generic-classes.ipynb) may be constrained by a type bound. 

- Type bounds limit the concrete values of the type variables and possibly reveal 
more information about the members of such types. 

- An _upper type bound_ or _type constraint_ **`T <: A` declares that type variable `T` refers to a subtype of type `A`**.

For instance:

In [ ]:
// Animals
abstract class Animal:
    def name: String

class Lion extends Animal:
  override def name: String = "Lion"

// Pets
abstract class Pet extends Animal

class Cat extends Pet:
  override def name: String = "Cat"

class Dog extends Pet:
  override def name: String = "Dog"

// Containers
class Container[A](x: A):
    def elem: A = x

class AnimalContainer[A <: Animal](x: A):
    def elem: A = x

class PetContainer[A <: Pet](x: A):
    def elem: A = x

In [ ]:
val lion = new Lion
val dog = new Dog
val cat = new Cat

Questions... 🖐️

- _Can I add the `lion` in `Container[Animal]`, why?_

In [ ]:
val container = Container[Animal](lion)

- _Can I add the `lion` in `PetContainer[Lion]`, why?_

In [ ]:
val petContainer = PetContainer[Lion](lion)

- _Can I add the `lion` and the `dog` in `AnimalContainer[Animal]`, why?_

In [ ]:
AnimalContainer[Animal](lion)

In [ ]:
AnimalContainer[Animal](dog)

- _Can I add the `dog` and the `cat` in `PetContainer[Cat]`, why?_

In [ ]:
PetContainer[Cat](dog)

In [ ]:
PetContainer[Cat](cat)

## Another example

**There are types and methods that, even if they work for a wide variety of types, 
they do not make sense for some types.** 

For example, the type `Set[T]`, **where `T` needs to be a type that can be compare to.**

Let's define the `Ordered[T]` trait. This definition allows values of type `T` to be comparable.

In [20]:
import scala.language.postfixOps

trait Ordered[A]:
    def compare(that: A): Int
    def < (that: A): Boolean = (this `compare` that) < 0
    def > (that: A): Boolean = (this `compare` that) > 0
    def <= (that: A): Boolean = (this `compare` that) <= 0
    def >= (that: A): Boolean = (this `compare` that) >= 0
    def compareTo(that: A): Int = compare(that)

import scala.language.postfixOps


defined trait Ordered

We use the trait `Ordered[T]` as a _type constraint_ in the generic 
type `Set[T]`, so any type `T` is subtype of `Ordered[T]`.

In [ ]:
import scala.language.postfixOps

abstract class Set[T <: Ordered[T]]:
    def incl(x: T): Set[T]
    def contains(x: T): Boolean

class EmptySet[T <: Ordered[T]] extends Set[T]:
    def contains(x: T): Boolean = false
    def incl(x: T): Set[T] = new NonEmptySet(x, new EmptySet[T], new EmptySet[T])

class NonEmptySet[T <: Ordered[T]](e: T, lset: Set[T], rset: Set[T]) extends Set[T]:
    def contains(x: T): Boolean =
        if (x < e) lset `contains` x // T needs to be comparable
        else if (x > e) rset `contains` x
        else true
    def incl(x: T): Set[T] =
        if (x < e) new NonEmptySet(e, lset `incl` x, rset)
        else if (x > e) new NonEmptySet(e, lset, rset `incl` x)
        else this

Finally, we define the type `Num` which implement the behaviour of an `Ordered[Num]` type.

In [ ]:
case class Num(value: Double) extends Ordered[Num] {
    def compare(that: Num): Int =
        if (this.value < that.value) -1
        else if (this.value > that.value) 1
        else 0
}

In [27]:
val s = new EmptySet[Num].incl(Num(9)).incl(Num(0)).incl(Num(10))

s: Set[Num] = ammonite.$sess.cmd22$Helper$NonEmptySet@6e85f200

In [26]:
s.contains(Num(0))

res26: Boolean = true

In [25]:
s.contains(Num(1001))

res25: Boolean = false

<p style="float: left;"><a href="polymorphic-methods.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="variances.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>